In [1]:
import pandas as pd
import os
file = ('6months Data.xls')
data = pd.read_excel(file)
comp = ["shell", "bp", "exxon mobil", "total", "cheveron", "conoco phillips", "conocophillips", "saudi aramco", "petrobras", "statoil", "petronas"]
output_path = 'output'
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [2]:
from datetime import datetime
try:
    today = datetime.now()
    d = today.replace(year=today.year-3)
    data = pd.DataFrame(data.query('`Run Date` >= @d'))
except KeyError:
    print('"Run Date" column is missing or present with a different name')

In [3]:
def size(s, df):
    try:
        df = pd.DataFrame(df.query('`Bit Size (in)` == @s'))
    except KeyError:
        print('"Bit Size (in)" column is missing or present with a different name')
    return df

In [4]:
def type(t, df):
    try:
        df['Bit Family'] = df['Bit Family'].fillna('blank')
        if (t=='PDC'):
            df = pd.DataFrame(df.query('`Bit Family` == "FCS" or `Bit Family` == "FCM"'))
        elif (t=='TCR'):
            df = pd.DataFrame(df.query('`Bit Family` == "RCT" or `Bit Family` == "RCI"'))
    except KeyError:
        print('"Bit Family" column is missing or present with a different name')
    return df

In [5]:
def depth(s, e, df):
    try:
        name = [col for col in df.columns if 'MD In' in col]
        name = name[0]
        df = pd.DataFrame(df.query('`{0}` >= @s'.format(name)))
        if (e!=-1): 
            df = pd.DataFrame(df.query('`{0}` <= @e'.format(name)))
    except KeyError:
        print('"MD In" column is missing or present with a different name')
    return df

In [6]:
def country(s, df):
    try:
        df['Country'] = df['Country'].fillna('blank')
        df = pd.DataFrame(df.query('Country == @s'))
    except:
        print('"Country" column is missing or present with a different name')
    return df

In [7]:
def company(c, df):
    try:
        df['Operator'] = df['Operator'].fillna('blank')
        df['Operator'] = df['Operator'].str.lower()
        df = df[df['Operator'].str.contains('|'.join(c), na=False)]
    except KeyError:
        print('"Operator" column is missing or present with a different name')
    return df

In [8]:
def meterage(df):
    name = [col for col in df.columns if 'Run Length' in col]
    try:
        name = name[0]
        index = df.columns.get_loc(name)
        df['Serial Number'] = df['Serial Number'].fillna('blank')
        df['cumm Meterage'] = df.groupby('Serial Number')[name].transform('cumsum')
        column = df.pop('cumm Meterage')
        df.insert(index+1, 'Cumm Meterage', column)
    except KeyError:
        print('"Run Length" column is missing or present with a different name')
    return name

In [9]:
def rop(df):
    name = [col for col in df.columns if 'ROP' in col]
    try:
        name = name[0]
    except KeyError:
        print('"ROP" column is missing or present with a different name')
    return name

In [10]:
def out(path, file, df, name):
    if not df.empty:
        df['Manufacturer'] = df['Manufacturer'].fillna('blank')
        unique_manufacturers = df['Manufacturer'].unique()

        with pd.ExcelWriter(path + '/' + file) as writer:
            for manufacturer in unique_manufacturers:
                group_df = df[df['Manufacturer'] == manufacturer]
                #if name != 1:
                    #group_df = group_df.sort_values(by=name, ascending=False)
                    #group_df = group_df.head(5)
                group_df.to_excel(writer, sheet_name=manufacturer, index=False)
    return

In [11]:
b1 = 'output/B.1'
if not os.path.exists(b1):
    os.makedirs(b1)

b10 = size(17.5, data)
b10 = type('TCR', b10)

b11 = depth(2000, -1, b10)
out(b1, 'B.1.1.xlsx', b11, 1)

b12 = company(comp, b10)
out(b1, 'B.1.2.xlsx', b12, 1)

b13 = country('India', b10)
out(b1, 'B.1.3.xlsx', b13, 1)

b141 = depth(2000, 3999, b10)
b141m = meterage(b141)
out(b1, 'B.1.4.1M.xlsx', b141, b141m)
b141r = rop(b141)
out(b1, 'B.1.4.1R.xlsx', b141, b141r)

b142 = depth(4000, 4999, b10)
b142m = meterage(b142)
out(b1, 'B.1.4.2M.xlsx', b142, b142m)
b142r = rop(b142)
out(b1, 'B.1.4.2R.xlsx', b142, b142r)


C:\Users\deeks\AppData\Local\Temp\ipykernel_18344\2978894701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Manufacturer'] = df['Manufacturer'].fillna('blank')


In [12]:
b2 = 'output/B.2'
if not os.path.exists(b2):
    os.makedirs(b2)

b20 = size(12.25, data)
b20 = type('TCR', b20)

b211 = depth(2000, 4000, b20)
out(b2, 'B.2.1.1.xlsx', b211, 1)
b212 = depth(4000, -1, b20)
out(b2, 'B.2.1.2.xlsx', b212, 1)

b22 = company(comp, b20)
out(b2, 'B.2.2.xlsx', b22, 1)

b23 = country('India', b20)
out(b2, 'B.2.3.xlsx', b23, 1)

b241 = depth(2000, 3999, b20)
b241m = meterage(b241)
out(b2, 'B.2.4.1M.xlsx', b241, b241m)
b241r = rop(b241)
out(b2, 'B.2.4.1R.xlsx', b241, b241r)

b242 = depth(4000, 4999, b20)
b242m = meterage(b242)
out(b2, 'B.2.4.2M.xlsx', b242, b242m)
b242r = rop(b242)
out(b2, 'B.2.4.2R.xlsx', b242, b242r)

b243 = depth(5000, -1, b20)
b243m = meterage(b243)
out(b2, 'B.2.4.3M.xlsx', b243, b243m)
b243r = rop(b243)
out(b2, 'B.2.4.3R.xlsx', b243, b243r)

C:\Users\deeks\AppData\Local\Temp\ipykernel_18344\2978894701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Manufacturer'] = df['Manufacturer'].fillna('blank')


In [13]:
b3 = 'output/B.3'
if not os.path.exists(b3):
    os.makedirs(b3)

b30 = size(8.5, data)
b30 = type('TCR', b30)

b311 = depth(2000, 4000, b30)
out(b3, 'B.3.1.1.xlsx', b311, 1)
b312 = depth(4000, 5000, b30)
out(b3, 'B.3.1.2.xlsx', b312, 1)
b313 = depth(5000, -1, b30)
out(b3, 'B.3.1.3.xlsx', b313, 1)

b32 = company(comp, b30)
out(b3, 'B.3.2.xlsx', b32, 1)

b33 = country('India', b30)
out(b3, 'B.3.3.xlsx', b33, 1)

b341 = depth(2000, 3999, b30)
b341m = meterage(b341)
out(b3, 'B.3.4.1M.xlsx', b341, b341m)
b341r = rop(b341)
out(b3, 'B.3.4.1R.xlsx', b341, b341r)

b342 = depth(4000, 4999, b30)
b342m = meterage(b342)
out(b3, 'B.3.4.2M.xlsx', b342, b342m)
b342r = rop(b342)
out(b3, 'B.3.4.2R.xlsx', b342, b342r)

b343 = depth(5000, -1, b30)
b343m = meterage(b343)
out(b3, 'B.3.4.3M.xlsx', b343, b343m)
b343r = rop(b343)
out(b3, 'B.3.4.3R.xlsx', b343, b343r)

In [14]:
b4 = 'output/B.4'
if not os.path.exists(b4):
    os.makedirs(b4)

s = [6, 5.875]
b40 = size(s, data)
b40 = type('TCR', b40)

b411 = depth(2000, 4000, b40)
out(b4, 'B.4.1.1.xlsx', b411, 1)
b412 = depth(4000, 5000, b40)
out(b4, 'B.4.1.2.xlsx', b412, 1)
b413 = depth(5000, -1, b40)
out(b4, 'B.4.1.3.xlsx', b413, 1)

b42 = company(comp, b40)
out(b4, 'B.4.2.xlsx', b42, 1)

b43 = country('India', b40)
out(b4, 'B.4.3.xlsx', b43, 1)

b441 = depth(2000, 3999, b40)
b441m = meterage(b441)
out(b4, 'B.4.4.1M.xlsx', b441, b441m)
b441r = rop(b441)
out(b4, 'B.4.4.1R.xlsx', b441, b441r)

b442 = depth(4000, 4999, b40)
b442m = meterage(b442)
out(b4, 'B.4.4.2M.xlsx', b442, b442m)
b442r = rop(b442)
out(b4, 'B.4.4.2R.xlsx', b442, b442r)

b443 = depth(5000, -1, b40)
b443m = meterage(b443)
out(b4, 'B.4.4.3M.xlsx', b443, b443m)
b443r = rop(b443)
out(b4, 'B.4.4.3R.xlsx', b443, b443r)

In [15]:
b5 = 'output/B.5'
if not os.path.exists(b5):
    os.makedirs(b5)

b50 = size(17.5, data)
b50 = type('PDC', b50)

b51 = depth(2000, -1, b50)
out(b5, 'B.5.1.xlsx', b51, 1)

b52 = company(comp, b50)
out(b5, 'B.5.2.xlsx', b52, 1)

b53 = country('India', b50)
out(b5, 'B.5.3.xlsx', b53, 1)

b541 = depth(2000, 3999, b50)
b541m = meterage(b541)
out(b5, 'B.5.4.1M.xlsx', b541, b541m)
b541r = rop(b541)
out(b5, 'B.5.4.1R.xlsx', b541, b541r)

b542 = depth(4000, 4999, b50)
b542m = meterage(b542)
out(b5, 'B.5.4.2M.xlsx', b542, b542m)
b542r = rop(b542)
out(b5, 'B.5.4.2R.xlsx', b542, b542r)

C:\Users\deeks\AppData\Local\Temp\ipykernel_18344\2978894701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Manufacturer'] = df['Manufacturer'].fillna('blank')


In [16]:
b6 = 'output/B.6'
if not os.path.exists(b6):
    os.makedirs(b6)

b60 = size(12.25, data)
b60 = type('PDC', b60)

b611 = depth(2000, 4000, b60)
out(b6, 'B.6.1.1.xlsx', b611, 1)
b612 = depth(4000, -1, b60)
out(b6, 'B.6.1.2.xlsx', b612, 1)

b62 = company(comp, b60)
out(b6, 'B.6.2.xlsx', b62, 1)

b63 = country('India', b60)
out(b6, 'B.6.3.xlsx', b63, 1)

b641 = depth(2000, 3999, b60)
b641m = meterage(b641)
out(b6, 'B.6.4.1M.xlsx', b641, b641m)
b641r = rop(b641)
out(b6, 'B.6.4.1R.xlsx', b641, b641r)

b642 = depth(4000, 4999, b60)
b642m = meterage(b642)
out(b6, 'B.6.4.2M.xlsx', b642, b642m)
b642r = rop(b642)
out(b6, 'B.6.4.2R.xlsx', b642, b642r)

b643 = depth(5000, -1, b60)
b643m = meterage(b643)
out(b6, 'B.6.4.3M.xlsx', b643, b643m)
b643r = rop(b643)
out(b6, 'B.6.4.3R.xlsx', b643, b643r)

C:\Users\deeks\AppData\Local\Temp\ipykernel_18344\2978894701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Manufacturer'] = df['Manufacturer'].fillna('blank')


In [17]:
b7 = 'output/B.7'
if not os.path.exists(b7):
    os.makedirs(b7)

b70 = size(8.5, data)
b70 = type('PDC', b70)

b711 = depth(2000, 4000, b70)
out(b7, 'B.7.1.1.xlsx', b711, 1)
b712 = depth(4000, 5000, b70)
out(b7, 'B.7.1.2.xlsx', b712, 1)
b713 = depth(5000, -1, b70)
out(b7, 'B.7.1.3.xlsx', b713, 1)

b72 = company(comp, b70)
out(b7, 'B.7.2.xlsx', b72, 1)

b73 = country('India', b70)
out(b7, 'B.7.3.xlsx', b73, 1)

b741 = depth(2000, 3999, b70)
b741m = meterage(b741)
out(b7, 'B.7.4.1M.xlsx', b741, b741m)
b741r = rop(b741)
out(b7, 'B.7.4.1R.xlsx', b741, b741r)

b742 = depth(4000, 4999, b70)
b742m = meterage(b742)
out(b7, 'B.7.4.2M.xlsx', b742, b742m)
b742r = rop(b742)
out(b7, 'B.7.4.2R.xlsx', b742, b742r)

b743 = depth(5000, -1, b70)
b743m = meterage(b743)
out(b7, 'B.7.4.3M.xlsx', b743, b743m)
b743r = rop(b743)
out(b7, 'B.7.4.3R.xlsx', b743, b743r)

C:\Users\deeks\AppData\Local\Temp\ipykernel_18344\2978894701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Manufacturer'] = df['Manufacturer'].fillna('blank')


In [18]:
b8 = 'output/B.8'
if not os.path.exists(b8):
    os.makedirs(b8)

s = [6, 5.875]
b80 = size(s, data)
b80 = type('PDC', b80)

b811 = depth(2000, 4000, b80)
out(b8, 'B.8.1.1.xlsx', b811, 1)
b812 = depth(4000, 5000, b80)
out(b8, 'B.8.1.2.xlsx', b812, 1)
b813 = depth(5000, -1, b80)
out(b8, 'B.8.1.3.xlsx', b813, 1)

b82 = company(comp, b80)
out(b8, 'B.8.2.xlsx', b82, 1)

b83 = country('India', b80)
out(b8, 'B.8.3.xlsx', b83, 1)

b841 = depth(2000, 3999, b80)
b841m = meterage(b841)
out(b8, 'B.8.4.1M.xlsx', b841, b841m)
b841r = rop(b841)
out(b8, 'B.8.4.1R.xlsx', b841, b841r)

b842 = depth(4000, 4999, b80)
b842m = meterage(b842)
out(b8, 'B.8.4.2M.xlsx', b842, b842m)
b842r = rop(b842)
out(b8, 'B.8.4.2R.xlsx', b842, b842r)

b843 = depth(5000, -1, b80)
b843m = meterage(b843)
out(b8, 'B.8.4.3M.xlsx', b843, b843m)
b843r = rop(b843)
out(b8, 'B.8.4.3R.xlsx', b843, b843r)